In [80]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import random
import lxml
import pandas as pd

In [83]:
# 출력 행 제한을 없애는 코드
pd.set_option('display.max_rows', 200)

### 날짜 생성 코드

In [29]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 오늘 날짜를 가져옵니다.
today = datetime.today()
# 10년 전의 날짜를 계산합니다.
ten_years_ago = today - relativedelta(years=10)

# 날짜 범위를 생성합니다.
dates = [ten_years_ago + relativedelta(days=i) for i in range((today - ten_years_ago).days)]

# 결과를 출력합니다.
for date in dates[::-1][:10]:
    print(date.strftime('%Y-%m-%d'))

2024-07-18
2024-07-17
2024-07-16
2024-07-15
2024-07-14
2024-07-13
2024-07-12
2024-07-11
2024-07-10
2024-07-09


In [30]:
# 크롬 옵션즈에 User-Agent, lang 같은 정보를 담아 셀레니움을 이용한 크롤링이 아닌 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')
options.add_argument("lang=ko_KR")

# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

driver.get("https://www.hanabank.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")  # 크롤링 할 웹사이트 주소

In [82]:
df_list = []
for idx, date in enumerate(dates[::-1][:3]):
    # 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
    wait = WebDriverWait(driver, 10)

    # 날짜 박스 #tmpInqStrDt
    datebox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
    datebox.clear()   # 기존에 입력된 값을 지웁니다.
    datebox.send_keys(f"{date.date()}")   # 새로운 값을 입력합니다.

    # 조회버튼   #HANA_CONTENTS_DIV > div.btnBoxCenter > a
    search_box = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
    search_box.click()   # 조회버튼 클릭
    # 생성한 시간 동안 대기합니다.
    time.sleep(random.uniform(3, 7))
    exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))

    # 페이지의 전체 HTML 소스를 가져옵니다.
    page_html = driver.page_source
    # soup = bs(page_html, "lxml")     # beautifulsoup으로 html을 parsing
    exchange_rate = pd.read_html(page_html)
    exchange_rate = exchange_rate[1]
    exchange_rate['date'] = f"{date.date()}"
    df_list.append(exchange_rate)
    print(f"전체 {len(dates)} 중 {idx+1:04d} 페이지 완료", end="\r")
# 리스트의 모든 DataFrame을 하나로 합칩니다.
final_df = pd.concat(df_list)

In [75]:
final_df['date'][0]

0    2024-07-18
0    2024-07-17
0    2024-07-16
Name: date, dtype: object

In [78]:
final_df[final_df['date'] == '2024-07-16']

통화       현찰                              송금          T/C 사실때  \
                 통화     사실 때            파실 때            보낼 때     받을 때 T/C 사실때   
                 통화       환율 Spread       환율 Spread     보낼 때     받을 때 T/C 사실때   
0            미국 USD  1405.16   1.75  1356.84   1.75  1394.50  1367.50     0.0   
1      일본 JPY (100)   899.59   1.75   868.65   1.75   892.78   875.46     0.0   
2            유로 EUR  1540.17   1.99  1480.07   1.99  1525.22  1495.02     0.0   
3            중국 CNY   199.40   5.00   180.42   5.00   191.80   188.02     0.0   
4            홍콩 HKD   180.37   1.97   173.41   1.97   178.65   175.13     0.0   
5            태국 THB    40.35   5.00    36.13   6.00    38.81    38.05     0.0   
6            대만 TWD    47.95  13.10    38.16  10.00     0.00     0.00     0.0   
7           필리핀 PHP    26.07  10.00    21.76   8.20    23.93    23.47     0.0   
8          싱가포르 SGD  1050.55   1.99  1009.57   1.99  1040.36  1019.76     0.0   
9            호주 AUD   947.01   1.97   910.43   1.97   938.00   919.44     0.0   
10    베트남 VND (100)     6.10  11.80     4.82  11.80     5.51     5.41     0.0   
11           영국 GBP  1831.51   1.97  1760.75   1.97  1814.09  1778.17     0.0   
12          캐나다 CAD  1028.64   1.97   988.90   1.97  1018.85   998.69     0.0   
13        말레이시아 MYR   318.04   7.40   274.22   7.40     0.00   293.17     0.0   
14          러시아 RUB    17.07   9.50    12.87  17.50    16.13    15.05     0.0   
15        남아공화국 ZAR    81.96   8.00    69.82   8.00    76.80    74.98     0.0   
16         노르웨이 NOK   132.92   3.30   121.35   5.70   129.96   127.40     0.0   
17         뉴질랜드 NZD   855.76   1.97   822.70   1.97   847.62   830.84     0.0   
18          덴마크 DKK   209.12   3.30   190.91   5.70   204.46   200.42     0.0   
19          멕시코 MXN    86.15  10.50    69.79  10.50    78.74    77.20     0.0   
20           몽골 MNT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
21          바레인 BHD  3897.98   6.40  3370.44   8.00  3700.15  3626.89     0.0   
22        방글라데시 BDT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
23          브라질 BRL   277.54  10.20   226.68  10.00     0.00   248.84     0.0   
24         브루나이 BND  1071.26   4.00   968.26   6.00     0.00     0.00     0.0   
25      사우디아라비아 SAR   391.38   6.30   342.79   6.90   371.87   364.51     0.0   
26         스리랑카 LKR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
27          스웨덴 SEK   135.20   3.30   123.43   5.70   132.19   129.59     0.0   
28          스위스 CHF  1593.70   1.97  1532.14   1.97  1578.54  1547.30     0.0   
29    아랍에미리트공화국 AED   396.66   5.50   350.05   6.90   379.74   372.24     0.0   
30          알제리 DZD     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
31           오만 OMR  3838.00   7.00  3335.84   7.00     0.00     0.00     0.0   
32          요르단 JOD  2121.16   8.90  1791.99   8.00     0.00     0.00     0.0   
33         이스라엘 ILS   417.60  10.00   349.27   8.00     0.00     0.00     0.0   
34          이집트 EGP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
35           인도 INR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
36  인도네시아 IDR (100)     9.46  10.00     7.74  10.00     8.68     8.52     0.0   
37           체코 CZK    64.83   8.50    54.39   9.00    60.41    59.11     0.0   
38           칠레 CLP     1.63  10.00     1.35  10.00     0.00     0.00     0.0   
39        카자흐스탄 KZT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
40          카타르 QAR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
41           케냐 KES     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
42         콜롬비아 COP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
43         쿠웨이트 KWD  4814.28   6.50  4158.83   8.00  4565.66  4475.26     0.0   
44         탄자니아 TZS     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
45         튀르키예 TRY     0.00   0.00     0.00   0.00    42.18    41.28     0.0   
46         파키스탄 PKR     0.00   0.00     0.00   0

In [84]:
final_df

통화       현찰                              송금          T/C 사실때  \
                 통화     사실 때            파실 때            보낼 때     받을 때 T/C 사실때   
                 통화       환율 Spread       환율 Spread     보낼 때     받을 때 T/C 사실때   
0            미국 USD  1410.25   1.75  1361.75   1.75  1399.50  1372.50     0.0   
1      일본 JPY (100)   896.00   1.75   865.18   1.75   889.21   871.97     0.0   
2            유로 EUR  1540.51   1.99  1480.41   1.99  1525.56  1495.36     0.0   
3            중국 CNY   199.95   5.00   180.91   5.00   192.33   188.53     0.0   
4            홍콩 HKD   180.95   1.97   173.97   1.97   179.23   175.69     0.0   
5            태국 THB    40.27   5.00    36.06   6.00    38.74    37.98     0.0   
6            대만 TWD    47.93  13.10    38.15  10.00     0.00     0.00     0.0   
7           필리핀 PHP    26.15  10.00    21.84   8.20    24.01    23.55     0.0   
8          싱가포르 SGD  1051.81   1.99  1010.77   1.99  1041.60  1020.98     0.0   
9            호주 AUD   947.83   1.97   911.21   1.97   938.81   920.23     0.0   
10    베트남 VND (100)     6.12  11.80     4.84  11.80     5.53     5.43     0.0   
11           영국 GBP  1829.45   1.97  1758.77   1.97  1812.05  1776.17     0.0   
12          캐나다 CAD  1031.04   1.97   991.22   1.97  1021.24  1001.02     0.0   
13        말레이시아 MYR   318.92   7.40   274.98   7.40     0.00   293.99     0.0   
14          러시아 RUB    17.16   9.50    12.94  17.50    16.22    15.14     0.0   
15        남아공화국 ZAR    81.95   8.00    69.81   8.00    76.79    74.97     0.0   
16         노르웨이 NOK   132.32   3.30   120.80   5.70   129.38   126.82     0.0   
17         뉴질랜드 NZD   854.48   1.97   821.48   1.97   846.35   829.61     0.0   
18          덴마크 DKK   209.13   3.30   190.92   5.70   204.47   200.43     0.0   
19          멕시코 MXN    85.21  10.50    69.03  10.50    77.89    76.35     0.0   
20           몽골 MNT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
21          바레인 BHD  3912.20   6.40  3382.73   8.00  3713.64  3640.12     0.0   
22        방글라데시 BDT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
23          브라질 BRL   273.89  10.20   223.69  10.00     0.00   245.56     0.0   
24         브루나이 BND  1072.54   4.00   969.42   6.00     0.00     0.00     0.0   
25      사우디아라비아 SAR   392.79   6.30   344.03   6.90   373.21   365.83     0.0   
26         스리랑카 LKR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
27          스웨덴 SEK   134.99   3.30   123.24   5.70   131.98   129.38     0.0   
28          스위스 CHF  1591.91   1.97  1530.41   1.97  1576.77  1545.55     0.0   
29    아랍에미리트공화국 AED   398.10   5.50   351.32   6.90   381.12   373.58     0.0   
30          알제리 DZD     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
31           오만 OMR  3852.00   7.00  3348.00   7.00     0.00     0.00     0.0   
32          요르단 JOD  2128.85   8.90  1798.49   8.00     0.00     0.00     0.0   
33         이스라엘 ILS   418.11  10.00   349.70   8.00     0.00     0.00     0.0   
34          이집트 EGP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
35           인도 INR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
36  인도네시아 IDR (100)     9.46  10.00     7.74  10.00     8.68     8.52     0.0   
37           체코 CZK    64.86   8.50    54.40   9.00    60.43    59.13     0.0   
38           칠레 CLP     1.61  10.00     1.33  10.00     0.00     0.00     0.0   
39        카자흐스탄 KZT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
40          카타르 QAR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
41           케냐 KES     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
42         콜롬비아 COP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
43         쿠웨이트 KWD  4829.18   6.50  4171.70   8.00  4579.79  4489.11     0.0   
44         탄자니아 TZS     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
45         튀르키예 TRY     0.00   0.00     0.00   0.00    42.36    41.44     0.0   
46         파키스탄 PKR     0.00   0.00     0.00   0

In [73]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, 0 to 57
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   (통화, 통화, 통화)                       174 non-null    object 
 1   (현찰, 사실 때, 환율)                     174 non-null    float64
 2   (현찰, 사실 때, Spread)                 174 non-null    float64
 3   (현찰, 파실 때, 환율)                     174 non-null    float64
 4   (현찰, 파실 때, Spread)                 174 non-null    float64
 5   (송금, 보낼 때, 보낼 때)                   174 non-null    float64
 6   (송금, 받을 때, 받을 때)                   174 non-null    float64
 7   (T/C 사실때, T/C 사실때, T/C 사실때)        174 non-null    float64
 8   (외화 수표 파실때, 외화 수표 파실때, 외화 수표 파실때)  174 non-null    float64
 9   (매매 기준율, 매매 기준율, 매매 기준율)           174 non-null    float64
 10  (환가 료율, 환가 료율, 환가 료율)              174 non-null    float64
 11  (미화 환산율, 미화 환산율, 미화 환산율)           174 non-null    float64
 12  